# PROGRAMMING QUESTION

## Packages

In [9]:
import pandas as pd
from datetime import datetime, timedelta

## 1.

In [26]:
# Sample to work with dataframe
player_lookup_data = {'player_id': [1, 2, 3],
                      'name': ['Patrick Murphy', 'Wander Suero', 'Kyle Finnegan'],
                      'throws': ['L', 'R', 'S']}

player_lookup = pd.DataFrame(player_lookup_data)

print(player_lookup)

   player_id            name throws
0          1  Patrick Murphy      L
1          2    Wander Suero      R
2          3   Kyle Finnegan      S


In [27]:
game_lookup_data = {'game_id': [1, 2, 3],
                    'game_date': ['2023-11-16', '2023-11-17', '2023-11-18'],
                    'home_team_id': ['a', 'b', 'c'],
                    'away_team_id': ['x', 'y', 'z']}

game_lookup = pd.DataFrame(game_lookup_data)

# Convert 'game_date' to datetime format
game_lookup['game_date'] = pd.to_datetime(game_lookup['game_date'])

print(game_lookup)


   game_id  game_date home_team_id away_team_id
0        1 2023-11-16            a            x
1        2 2023-11-17            b            y
2        3 2023-11-18            c            z


In [28]:
pitches_data = {'game_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
                 'pitch_no': [1, 2, 3, 1, 2, 3, 1, 2, 3],
                 'at_bat_no': [1, 1, 1, 2, 2, 2, 3, 3, 3],
                 'pitch_of_at_bat': [1, 2, 3, 1, 2, 3, 1, 2, 3],
                 'inning': [1, 2, 3, 1, 2, 3, 1, 2, 3],
                 'top_inning': [1, 1, 1, 0, 0, 0, 1, 1, 1],
                 'pitcher_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
                 'batter_id': [101, 102, 103, 201, 202, 203, 301, 302, 303],
                 'velo': [90.1, 91.2, 92.0, 88.5, 87.7, 89.3, 95.0, 94.2, 93.8]}

pitches = pd.DataFrame(pitches_data)

print(pitches)


   game_id  pitch_no  at_bat_no  pitch_of_at_bat  inning  top_inning  \
0        1         1          1                1       1           1   
1        1         2          1                2       2           1   
2        1         3          1                3       3           1   
3        2         1          2                1       1           0   
4        2         2          2                2       2           0   
5        2         3          2                3       3           0   
6        3         1          3                1       1           1   
7        3         2          3                2       2           1   
8        3         3          3                3       3           1   

   pitcher_id  batter_id  velo  
0           1        101  90.1  
1           1        102  91.2  
2           1        103  92.0  
3           2        201  88.5  
4           2        202  87.7  
5           2        203  89.3  
6           3        301  95.0  
7           3  

In [29]:
# Merge the dataframes
merged_data = pd.merge(pitches, game_lookup, on='game_id')
merged_data = pd.merge(merged_data, player_lookup[['player_id', 'name']], left_on='pitcher_id', right_on='player_id')

# Filter data for the last 7 days
last_7_days = game_lookup['game_date'].max() - pd.DateOffset(days=6)
filtered_data = merged_data[merged_data['game_date'] >= last_7_days]

# Group by pitcher_id and calculate statistics
pitcher_summary = filtered_data.groupby(['pitcher_id', 'name']).agg(
    n_games=('game_id', 'nunique'),
    n_pitches=('pitch_no', 'count'),
    velo_avg=('velo', 'mean')
).reset_index()

print(pitcher_summary)

   pitcher_id            name  n_games  n_pitches   velo_avg
0           1  Patrick Murphy        1          3  91.100000
1           2    Wander Suero        1          3  88.500000
2           3   Kyle Finnegan        1          3  94.333333


## 2.

In [25]:
# The data
data = {
    "Name": ["Patrick Murphy", "Wander Suero", "Kyle Finnegan", "Austin Voth", "Ryan Harper", "Mason Thompson", "Sam Clay", "Alberto Balnado"],
    "9/10/21": ["0.1 | 1", "", "", "", "", "", "", "0.1 | 6"],
    "9/11/21": ["", "", "", "", "1.0 | 19", "", "1.0 | 16", "0.1 | 13"],
    "9/12/21": ["", "", "1.2 | 17", "", "", "0.1 | 17", "", ""],
    "9/13/21": ["1.0 | 14", "", "", "1.0 | 19", "", "", "1.0 | 13", ""],
    "9/14/21": ["", "1.0 | 10", "", "1.0 | 17", "", "", "", "1.0 | 12"],
    "9/15/21": ["", "0.2 | 11", "1.1 | 22", "", "", "0.2 | 6", "0.1 | 14", "1.0 | 17"],
    "9/16/21": ["", "","", "", "", "", "", ""],
    "Last 3 days": ["0.0 | 0", "1.2 | 21", "1.1 | 22", "1.0 | 17", "0.0 | 0", "0.2 | 6", "0.1 | 14", "2.0 | 29"],
    "Last 7 days": ["1.1 | 27","1.2 | 21", "3.0 | 39", "2.0 | 36", "1.0 | 19", "1.0 | 23", "2.1 | 43", "2.2 | 48"]
}

df = pd.DataFrame(data)

# Extract relevant columns for processing
pitch_data = df.drop(columns=['Last 3 days', 'Last 7 days'])

# Melt the DataFrame to convert it to a long format
melted_df = pitch_data.melt(id_vars='Name', var_name='Date', value_name='Pitch_Count')

# Filter out rows with empty strings in 'Pitch_Count'
melted_df = melted_df[melted_df['Pitch_Count'] != '']

# Split the 'Pitch_Count' column into 'Innings' and 'Pitch_Count'
melted_df[['Innings', 'Pitch_Count']] = melted_df['Pitch_Count'].str.split('|', expand=True)
melted_df['Pitch_Count'] = melted_df['Pitch_Count'].astype(int)

# Create a pivot table for pitch count by pitcher and date
pitch_count_matrix = melted_df.pivot_table(values='Pitch_Count', index='Name', columns='Date', aggfunc='sum', fill_value=0)

# Calculate total pitch count for the last 3 days and last 7 days
pitch_count_matrix['Total Last 3 Days'] = pitch_count_matrix.iloc[:, -4:-1].sum(axis=1)
pitch_count_matrix['Total Last 7 Days'] = pitch_count_matrix.iloc[:, :-1].sum(axis=1)

# Display the resulting DataFrame
print(pitch_count_matrix)

Date             9/10/21  9/11/21  9/12/21  9/13/21  9/14/21  9/15/21  \
Name                                                                    
Alberto Balnado        6       13        0        0       12       17   
Austin Voth            0        0        0       19       17        0   
Kyle Finnegan          0        0       17        0        0       22   
Mason Thompson         0        0       17        0        0        6   
Patrick Murphy         1        0        0       14        0        0   
Ryan Harper            0       19        0        0        0        0   
Sam Clay               0       16        0       13        0       14   
Wander Suero           0        0        0        0       10       11   

Date             Total Last 3 Days  Total Last 7 Days  
Name                                                   
Alberto Balnado                 12                 48  
Austin Voth                     36                 36  
Kyle Finnegan                   17           

In [24]:
# Extract relevant columns for processing
pitch_data = df.drop(columns=['Last 3 days', 'Last 7 days'])

# Melt the DataFrame to convert it to a long format
melted_df = pitch_data.melt(id_vars='Name', var_name='Date', value_name='Pitch_Count')

# Filter out rows with empty strings in 'Pitch_Count'
melted_df = melted_df[melted_df['Pitch_Count'] != '']

# Split the 'Pitch_Count' column into 'Innings' and 'Pitch_Count'
melted_df[['Innings', 'Pitch_Count']] = melted_df['Pitch_Count'].str.split('|', expand=True)
melted_df['Pitch_Count'] = melted_df['Pitch_Count'].astype(int)

# Create a pivot table for pitch count by pitcher and date
pitch_count_matrix = melted_df.pivot_table(values='Pitch_Count', index='Name', columns='Date', aggfunc='sum', fill_value=0)

# Calculate total pitch count for the last 3 days and last 7 days
pitch_count_matrix['Total Last 3 Days'] = pitch_count_matrix.iloc[:, -4:-1].sum(axis=1)
pitch_count_matrix['Total Last 7 Days'] = pitch_count_matrix.iloc[:, :-1].sum(axis=1)

# Display the resulting DataFrame
print(pitch_count_matrix)

KeyError: "['Last 3 days', 'Last 7 days'] not found in axis"